In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows',None)

In [ ]:
ipdata=pd.read_csv("output.txt",sep='\t',header=None,names=['file_name','label','start','end','content','normal'],na_filter=False)
ipdata=ipdata.fillna("NULL")

PHI=frozenset({"PATIENT","DOCTOR","USERNAME","PROFESSION","ROOM","DEPARTMENT","HOSPITAL","ORGANIZATION","STREET","CITY","STATE",
        "COUNTRY","ZIP","LOCATION-OTHER","AGE","DATE","TIME","DURATION","SET","PHONE","FAX","EMAIL","URL","IPADDR","SSN",
        "MEDICALRECORD","HEALTHPLAN","ACCOUNT","LICENSE","VECHICLE","DEVICE","BIOID","IDNUM","OTHER"})

for i in range(len(ipdata)):
   if (ipdata.loc[i]["content"]=="NULL"):
    ipdata=ipdata.drop(i)
   elif (ipdata.loc[i]["label"] not in PHI):
    ipdata=ipdata.drop(i)
   elif(len(ipdata.loc[i]["content"]) > 100 or len(ipdata.loc[i]["normal"]) > 100):
    ipdata=ipdata.drop(i)

ipdata = ipdata.reset_index(drop=True)

for i in range(2,len(ipdata)):
  if(ipdata.loc[i]["label"] == "ZIP" and ipdata.loc[i-1]["label"] == "STATE" and ipdata.loc[i-2]["label"] == "CITY"):
    if(int(ipdata.loc[i-1]["start"]) < int(ipdata.loc[i-2]["end"])):
      ipdata.loc[i-1,"start"] = int(ipdata.loc[i-2]["end"])+2
      ipdata.loc[i-1,"end"] = int(ipdata.loc[i-1]["start"])+len(ipdata.loc[i]["content"])
    if(int(ipdata.loc[i]["start"]) < int(ipdata.loc[i-1]["end"])):
      ipdata.loc[i,"start"] = int(ipdata.loc[i-1]["end"])+2
      ipdata.loc[i,"end"] = int(ipdata.loc[i]["start"])+len(ipdata.loc[i]["content"])

end = 0
i=1

while(i!=len(ipdata)-1):
   if (ipdata.loc[i]["file_name"] == ipdata.loc[i-1]["file_name"]):
    if(int(ipdata.loc[i]["start"]) > end):
      end = int(ipdata.loc[i]["end"])
      i+=1
    else:
      ipdata=ipdata.drop(i)
      ipdata = ipdata.reset_index(drop=True)
   else:
    if(ipdata.loc[i]["start"] >= 0):
      end = ipdata.loc[i]["end"]
      i+=1
    else:
      ipdata=ipdata.drop(i)
      ipdata = ipdata.reset_index(drop=True)
ipdata = ipdata.reset_index(drop=True)

ipdata.replace('NULL', np.nan, inplace=True)
ipdata.to_csv("answer.txt", sep='\t', index=False, header=False)